## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-10-21-51-03 +0000,wapo,Fox News host Will Cain emotional as he announ...,https://www.washingtonpost.com
1,2025-09-10-21-51-03 +0000,wapo,Live updates: Conservative activist Charlie Ki...,https://www.washingtonpost.com/nation/2025/09/...
2,2025-09-10-21-50-36 +0000,nyt,What to Know About the Mass Protests in Nepal,https://www.nytimes.com/video/world/asia/10000...
3,2025-09-10-21-49-05 +0000,startribune,Charlie Kirk was slated to come to the Univers...,https://www.startribune.com/charlie-kirk-unive...
4,2025-09-10-21-47-31 +0000,nypost,"Who is Erika Frantzve, Charlie Kirk’s wife and...",https://nypost.com/us-news/who-is-erika-frantzve/


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,58
74,new,25
8,kirk,23
15,charlie,21
154,israel,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
45,2025-09-10-20-37-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...,144
97,2025-09-10-19-14-34 +0000,nypost,Trump says conservative activist Charlie Kirk ...,https://nypost.com/2025/09/10/us-news/trump-re...,144
367,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...,138
91,2025-09-10-19-21-19 +0000,cbc,Conservative U.S. commentator Charlie Kirk has...,https://www.cbc.ca/news/world/charlie-kirk-sho...,133
46,2025-09-10-20-36-13 +0000,nyt,Trump and Leaders Across Political Spectrum Re...,https://www.nytimes.com/2025/09/10/us/charlie-...,132


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
45,144,2025-09-10-20-37-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...
148,106,2025-09-10-17-08-00 +0000,wsj,Former Vice President Kamala Harris said in a ...,https://www.wsj.com/politics/elections/harris-...
1,97,2025-09-10-21-51-03 +0000,wapo,Live updates: Conservative activist Charlie Ki...,https://www.washingtonpost.com/nation/2025/09/...
206,81,2025-09-10-13-57-09 +0000,wapo,Israel escalates attacks on Gaza City as Qatar...,https://www.washingtonpost.com/world/2025/09/1...
256,59,2025-09-10-10-00-00 +0000,nypost,Lisa Cook should be kicked out of Fed’s Septem...,https://nypost.com/2025/09/10/us-news/lisa-coo...
335,51,2025-09-10-00-36-34 +0000,nypost,Poland’s airspace ‘repeatedly violated’ by Rus...,https://nypost.com/2025/09/09/world-news/ukrai...
190,50,2025-09-10-15-04-00 +0000,wsj,If the Supreme Court rules in favor of the pre...,https://www.wsj.com/economy/trade/tariff-case-...
239,45,2025-09-10-11-33-00 +0000,wsj,"Rep. Jerry Nadler of New York, age 78, said he...",https://www.wsj.com/politics/democrats-retirem...
36,44,2025-09-10-20-48-55 +0000,nypost,Speaker Johnson announces moment of silence fo...,https://nypost.com/2025/09/10/us-news/speaker-...
15,42,2025-09-10-21-22-04 +0000,wapo,France welcomes new prime minister with ‘block...,https://www.washingtonpost.com/world/2025/09/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
